In [1]:
# Importing necessary libraries
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

2025-08-04 11:12:40.711952: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754286160.724487  278324 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754286160.728424  278324 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754286160.739614  278324 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754286160.739626  278324 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754286160.739627  278324 computation_placer.cc:177] computation placer alr

In [88]:
# Importing data
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [3]:
# LiL EDA
class_dist = [0]*10
for x in y_train:
    class_dist[x]+=1
print(class_dist)
plt.imshow(x_train[1])

[6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000]

In [89]:
x_train = np.expand_dims(x_train,-1)
x_test = np.expand_dims(x_test,-1)

In [58]:
def preprocess(image,labels):
    image = tf.image.grayscale_to_rgb(image)/255
    image = tf.image.resize_with_pad(
        image,
        96,
        96,
    )
    labels = tf.one_hot(labels,depth = 10)
    return image,labels

In [90]:
# Preprocess data
tf_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train))
test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test))
tf_ds = tf_ds.map(preprocess)
tf_ds = tf_ds.batch(32).prefetch(tf.data.experimental.AUTOTUNE)
test_ds = test_ds.map(preprocess)
test_ds = test_ds.batch(32).prefetch(tf.data.experimental.AUTOTUNE)

In [77]:
# Importing model
MobileNetV2 = tf.keras.applications.MobileNetV2(
    input_shape=(96,96,3),
    alpha=1.0,
    include_top=False,
    weights='imagenet',
)
# Model redying for tranfer learning
for layer in MobileNetV2.layers:
    layer.trainable = False

<InputLayer name=input_layer, built=True>
<Conv2D name=Conv1, built=True>
<BatchNormalization name=bn_Conv1, built=True>
<ReLU name=Conv1_relu, built=True>
<DepthwiseConv2D name=expanded_conv_depthwise, built=True>
<BatchNormalization name=expanded_conv_depthwise_BN, built=True>
<ReLU name=expanded_conv_depthwise_relu, built=True>
<Conv2D name=expanded_conv_project, built=True>
<BatchNormalization name=expanded_conv_project_BN, built=True>
<Conv2D name=block_1_expand, built=True>
<BatchNormalization name=block_1_expand_BN, built=True>
<ReLU name=block_1_expand_relu, built=True>
<ZeroPadding2D name=block_1_pad, built=True>
<DepthwiseConv2D name=block_1_depthwise, built=True>
<BatchNormalization name=block_1_depthwise_BN, built=True>
<ReLU name=block_1_depthwise_relu, built=True>
<Conv2D name=block_1_project, built=True>
<BatchNormalization name=block_1_project_BN, built=True>
<Conv2D name=block_2_expand, built=True>
<BatchNormalization name=block_2_expand_BN, built=True>
<ReLU name=bloc

In [82]:
# Adding the new head to the model
X = MobileNetV2.output
X = tf.keras.layers.GlobalAveragePooling2D()(X)
X = tf.keras.layers.Dense(128, activation='relu')(X)   # optional intermediate layer
outputs = tf.keras.layers.Dense(10, activation='softmax')(X)  # for 10 classes like CIFAR10/FashionMNIST

# Create new model
model = tf.keras.Model(inputs=MobileNetV2.input, outputs=outputs)

In [84]:
# Compile and train the model
model.compile(optimizer = "adam",loss = tf.keras.losses.CategoricalCrossentropy(),metrics = ["accuracy"])
model.fit(tf_ds,epochs = 10)

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 83s 43ms/step - accuracy: 0.8365 - loss: 0.4574
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 80s 43ms/step - accuracy: 0.8965 - loss: 0.2800
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 101s 54ms/step - accuracy: 0.9080 - loss: 0.2433
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 92s 49ms/step - accuracy: 0.9186 - loss: 0.2158
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 79s 42ms/step - accuracy: 0.9294 - loss: 0.1909
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 79s 42ms/step - accuracy: 0.9356 - loss: 0.1731
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 80s 43ms/step - accuracy: 0.9410 - loss: 0.1557
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 44ms/step - accuracy: 0.9486 - loss: 0.1396
Epoch 9/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 85s 45ms/step - accuracy: 0.9530 - loss: 0.1271
Epoch 10/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 85s 46ms/step - accuracy: 0.9565 - loss: 0.1164


In [91]:
# Evaluate the model
model.evaluate(test_ds)

313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 43ms/step - accuracy: 0.8830 - loss: 0.4550


[0.45367249846458435, 0.8848999738693237]